In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import seaborn as sns
import numpy as np
import glob
from pathlib import Path
from experiment_utils import Experiment, ExperimentFilter, Plotting

In [ ]:
experiments = Experiment.get_experiments()
experiments = list(filter(ExperimentFilter.by_experiment_name('ftalat'), experiments))
experiment = ExperimentFilter.get_latest(experiments)
experiment

In [ ]:
# us -> MHz
reference_frequency_per_time_unit = 2.0E3
TRANSITION_LATENCY = "Transition latency [µs]"
WAIT_LATENCY = "Time since last frequency change [us]"
WAIT_LATENCY_BETWEEN_RUNS = "Time between detection of last frequency\nchange from source to target frequency [µs]"
SOURCE_FREQUENCY = "Source Frequency [GHz]"
TARGET_FREQUENCY = "Target Frequency [GHz]"
FREQ_TRANSITION = "Frequency transition [GHz]"

In [ ]:
processed_data = pd.DataFrame()
all_data = pd.DataFrame()

for file in glob.glob(f'{experiment.path}/*.txt'):
    start, target = Path(file).stem.split('_')
    start_ghz = int(start) / 1e6
    target_ghz = int(target) / 1e6

    # filter out too high turbo frequencies
    if target_ghz > 2.9 or start_ghz > 2.9:
        continue

    try:
        data = pd.read_csv(file, sep='\t',comment='#')
        print(data)
    except:
        continue
    data[TRANSITION_LATENCY] = data["Change time (with write) [cycles]"] / reference_frequency_per_time_unit
    data[WAIT_LATENCY] = data["Time since last frequency change [cycles]"] / reference_frequency_per_time_unit
    data["Change time [us]"] = data["Change time [cycles]"] / reference_frequency_per_time_unit
    data["Change time (with write) [us]"] = data["Change time (with write) [cycles]"] / reference_frequency_per_time_unit
    data["Write cost [us]"] = data["Write cost [cycles]"] / reference_frequency_per_time_unit
    data["second_change [us]"] = (data["Time since last frequency change request [cycles]"] - data["Time since last frequency change [cycles]"] + data["Change time (with write) [cycles]"]) / reference_frequency_per_time_unit
    data["second_validation_interval [us]"] = (data["Time since last frequency change [cycles]"] - data["Change time (with write) [cycles]"]) / reference_frequency_per_time_unit - data["Wait time [us]"]
    data[WAIT_LATENCY_BETWEEN_RUNS] = data["Detected frequency change timestamp [cycles]"].diff() / reference_frequency_per_time_unit

    # filter invalid measurement values
    data=data[data[TRANSITION_LATENCY]>0]
    # discard the first row
    data = data.iloc[1:]

    processed_data = pd.concat([processed_data, pd.DataFrame([
        {'valids': len(data),
         'median_<1ms': np.median(data[data[WAIT_LATENCY] < 1000][TRANSITION_LATENCY].quantile(0.5)),
         'median_>1ms': np.median(data[data[WAIT_LATENCY] > 1000][TRANSITION_LATENCY].quantile(0.5)),
         'median': np.median(data[TRANSITION_LATENCY].quantile(0.5)),
         '1percentQuantile': data[TRANSITION_LATENCY].quantile(0.01),
         '95percentQuantile': data[TRANSITION_LATENCY].quantile(0.95),
         '99percentQuantile': data[TRANSITION_LATENCY].quantile(0.99),
         SOURCE_FREQUENCY: start_ghz,
         TARGET_FREQUENCY: target_ghz}
    ])])

    data[SOURCE_FREQUENCY] = start_ghz
    data[TARGET_FREQUENCY] = target_ghz
    data[FREQ_TRANSITION] = f"{start_ghz}GHz -> {target_ghz}GHz"
    all_data = pd.concat([all_data, data])

processed_data

In [ ]:
all_data.reset_index()
all_data

In [ ]:
all_data_random_sample = all_data.sample(frac=.10)
all_data_random_sample

In [ ]:
plt.rcParams['figure.figsize'] = 5,5

ax = sns.heatmap(data=processed_data.pivot(index=SOURCE_FREQUENCY, columns=TARGET_FREQUENCY, values="valids"), vmin=0, square=True, cmap="viridis", linecolor="black", linewidths=0.5)
ax.invert_yaxis()

Plotting.savefig(experiment, 'ftalat_valids_hati.pdf', annotations_y_offset=0.05, annotations_y_spacing=0.025)

In [ ]:
plt.rcParams['figure.figsize'] = 5,5

ax = sns.heatmap(data=processed_data.pivot(index=SOURCE_FREQUENCY, columns=TARGET_FREQUENCY, values="median"), vmin=0, vmax=1200, square=True, cmap="viridis", linecolor="black", linewidths=0.5)
ax.invert_yaxis()

Plotting.savefig(experiment, 'ftalat_median_hati.pdf', annotations_y_offset=0.05, annotations_y_spacing=0.025)

In [ ]:
plt.rcParams['figure.figsize'] = 5,5

ax = sns.heatmap(data=processed_data.pivot(index=SOURCE_FREQUENCY, columns=TARGET_FREQUENCY, values="median_<1ms"), vmin=0, vmax=1200, square=True, cmap="viridis", linecolor="black", linewidths=0.5)
ax.invert_yaxis()

Plotting.savefig(experiment, 'ftalat_median_<1ms_hati.pdf', annotations_y_offset=0.05, annotations_y_spacing=0.025)

In [ ]:
plt.rcParams['figure.figsize'] = 5,5

ax = sns.heatmap(data=processed_data.pivot(index=SOURCE_FREQUENCY, columns=TARGET_FREQUENCY, values="median_>1ms"), vmin=0, vmax=1200, square=True, cmap="viridis", linecolor="black", linewidths=0.5)
ax.invert_yaxis()

Plotting.savefig(experiment, 'ftalat_median_>1ms_hati.pdf', annotations_y_offset=0.05, annotations_y_spacing=0.025)

In [ ]:
plt.rcParams['figure.figsize'] = 5,5

ax = sns.heatmap(data=processed_data.pivot(index=SOURCE_FREQUENCY, columns=TARGET_FREQUENCY, values="1percentQuantile"), vmin=0, square=True, cmap="viridis", linecolor="black", linewidths=0.5)
ax.invert_yaxis()

Plotting.savefig(experiment, 'ftalat_1percentQuantile_hati.pdf', annotations_y_offset=0.05, annotations_y_spacing=0.025)

In [ ]:
plt.rcParams['figure.figsize'] = 5,5

ax = sns.heatmap(data=processed_data.pivot(index=SOURCE_FREQUENCY, columns=TARGET_FREQUENCY, values="95percentQuantile"), vmin=0, square=True, cmap="viridis", linecolor="black", linewidths=0.5)
ax.invert_yaxis()

Plotting.savefig(experiment, 'ftalat_95percentQuantile_hati.pdf', annotations_y_offset=0.05, annotations_y_spacing=0.025)

In [ ]:
plt.rcParams['figure.figsize'] = 5,5

ax = sns.heatmap(data=processed_data.pivot(index=SOURCE_FREQUENCY, columns=TARGET_FREQUENCY, values="99percentQuantile"), vmin=0, square=True, cmap="viridis", linecolor="black", linewidths=0.5)
ax.invert_yaxis()

Plotting.savefig(experiment, 'ftalat_99percentQuantile_hati.pdf', annotations_y_offset=0.05, annotations_y_spacing=0.025)

In [ ]:
col_order = [1.8, 1.9, 2.0, 2.1, 2.2, 2.3, 2.4, 2.5, 2.6, 2.7, 2.8, 2.9]

In [ ]:
filtered_data = all_data
filtered_data = filtered_data[filtered_data[SOURCE_FREQUENCY] == 1.2]
filtered_data = filtered_data[filtered_data[TARGET_FREQUENCY] >= 1.8]
filtered_data = filtered_data[filtered_data[TARGET_FREQUENCY] <= 2.9]
filtered_data

In [ ]:
g = sns.FacetGrid(filtered_data, col=TARGET_FREQUENCY,  row=SOURCE_FREQUENCY, height=5, col_order=col_order)
g.map(sns.histplot, "Change time [us]", alpha=0.3)
g.add_legend()

In [ ]:
g = sns.FacetGrid(filtered_data, col=TARGET_FREQUENCY,  row=SOURCE_FREQUENCY, height=5, col_order=col_order)
g.map(sns.histplot, "Wait time [us]", alpha=0.3, binwidth=100)
g.add_legend()

In [ ]:
g = sns.FacetGrid(filtered_data, col=TARGET_FREQUENCY, height=5, col_order=col_order, ylim=(0,2000), xlim=(0, 20000), col_wrap=4)
g.map(sns.scatterplot, WAIT_LATENCY_BETWEEN_RUNS, TRANSITION_LATENCY, alpha=0.2)
g.add_legend()
buckets = [2000+ 1625*i for i in range(11)]
for axi in g.axes:
    for i in buckets:
        axi.axvline(x=i, color='black', dashes=[2], linewidth=1)
    axi.axhline(y=900, color='black', dashes=[2], linewidth=1)
    axi.axhline(y=900+200, color='black', dashes=[2], linewidth=1)
    axi.axhline(y=900+400, color='black', dashes=[2], linewidth=1)
    axi.axhline(y=900+600, color='black', dashes=[2], linewidth=1)
    axi.axhline(y=900+800, color='black', dashes=[2], linewidth=1)
    axi.axhline(y=900-200, color='black', dashes=[2], linewidth=1)
    axi.axhline(y=900-400, color='black', dashes=[2], linewidth=1)
    axi.axhline(y=900-600, color='black', dashes=[2], linewidth=1)
    axi.axhline(y=900-800, color='black', dashes=[2], linewidth=1)

Plotting.savefig(experiment, 'ftalat_scatter_wait_transition_latency_hati.pdf', annotations_y_offset=0, annotations_y_spacing=0.0125)

In [ ]:
g = sns.FacetGrid(filtered_data, col=TARGET_FREQUENCY, height=5, col_order=col_order, ylim=(0,2000), xlim=(0, 20000), col_wrap=4)
g.map(sns.scatterplot, WAIT_LATENCY, TRANSITION_LATENCY, alpha=0.2)
g.add_legend()
buckets = [2000, 3500, 5000, 6500]
for axi in g.axes:
    for i in buckets:
        axi.axvline(x=i, color='black', dashes=[2], linewidth=1)
    axi.axhline(y=875, color='black', dashes=[2], linewidth=1)

In [ ]:
for source_freq in [0.8, 0.9, 1.0, 1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2.0, 2.1, 2.2, 2.3, 2.4, 2.5, 2.6, 2.7, 2.8, 2.9]:
    filtered_data = all_data
    filtered_data = filtered_data[filtered_data[SOURCE_FREQUENCY] == source_freq]

    g = sns.FacetGrid(filtered_data, col=TARGET_FREQUENCY, height=5, ylim=(0,2000), xlim=(0, 20000), col_wrap=4)
    g.map(sns.scatterplot, WAIT_LATENCY_BETWEEN_RUNS, TRANSITION_LATENCY, alpha=0.2)
    g.add_legend()
    buckets = [2000+ 1625*i for i in range(11)]
    for axi in g.axes:
        for i in buckets:
            axi.axvline(x=i, color='black', dashes=[2], linewidth=1)
        axi.axhline(y=900, color='black', dashes=[2], linewidth=1)
        axi.axhline(y=900+200, color='black', dashes=[2], linewidth=1)
        axi.axhline(y=900+400, color='black', dashes=[2], linewidth=1)
        axi.axhline(y=900+600, color='black', dashes=[2], linewidth=1)
        axi.axhline(y=900+800, color='black', dashes=[2], linewidth=1)
        axi.axhline(y=900-200, color='black', dashes=[2], linewidth=1)
        axi.axhline(y=900-400, color='black', dashes=[2], linewidth=1)
        axi.axhline(y=900-600, color='black', dashes=[2], linewidth=1)
        axi.axhline(y=900-800, color='black', dashes=[2], linewidth=1)

    Plotting.savefig(experiment, f'ftalat_scatter_wait_transition_latency_hati_source_{source_freq}.pdf', annotations_y_offset=0, annotations_y_spacing=0.005)